In [1]:
import pandas as pd
df = pd.read_csv('../checkpoint 1/preprocessed_fakeReviewData.csv')

In [2]:
df.head()

,category,rating,label,text_,vector
0,Home_and_Kitchen_5,5.0,CG,"['love', 'well', 'make', 'sturdy', 'comfortabl...",[ 0.51234096 -0.8662119 -0.3682067 0.848585...
1,Home_and_Kitchen_5,5.0,CG,"['love', 'great', 'upgrade', 'original', 'ive'...",[-0.32024422 -0.35106233 -0.23759088 0.930493...
2,Home_and_Kitchen_5,5.0,CG,"['pillow', 'save', 'back', 'love', 'look', 'fe...",[ 1.0715518 -0.32561246 -1.3107281 0.556225...
3,Home_and_Kitchen_5,1.0,CG,"['miss', 'information', 'use', 'great', 'produ...",[ 0.04733269 -0.03660731 -0.6258498 0.412645...
4,Home_and_Kitchen_5,5.0,CG,"['nice', 'set', 'good', 'quality', 'set', 'two...",[ 0.11883444 -0.46801132 -0.70364386 1.015384...


In [3]:
df['vector'].head()

0    [ 0.51234096 -0.8662119  -0.3682067   0.848585...
1    [-0.32024422 -0.35106233 -0.23759088  0.930493...
2    [ 1.0715518  -0.32561246 -1.3107281   0.556225...
3    [ 0.04733269 -0.03660731 -0.6258498   0.412645...
4    [ 0.11883444 -0.46801132 -0.70364386  1.015384...
Name: vector, dtype: object

In [4]:
df.shape

(40419, 5)

In [5]:
def string_to_np_array(input_string):
    # Remove square brackets and newlines, then split by space
    cleaned_string = input_string.replace('[', '').replace(']', '').replace('\n', ' ')
    # Convert the cleaned string into a numpy array of floats
    np_array = np.fromstring(cleaned_string, dtype=float, sep=' ')
    return np_array

In [6]:
import numpy as np
df['vector'] = df['vector'].apply(string_to_np_array)

In [7]:
df['vector'].head()

0    [0.51234096, -0.8662119, -0.3682067, 0.8485853...
1    [-0.32024422, -0.35106233, -0.23759088, 0.9304...
2    [1.0715518, -0.32561246, -1.3107281, 0.5562251...
3    [0.04733269, -0.03660731, -0.6258498, 0.412645...
4    [0.11883444, -0.46801132, -0.70364386, 1.01538...
Name: vector, dtype: object

In [8]:
X = np.vstack(df['vector'].to_numpy())

In [9]:
X.shape

(40419, 100)

In [10]:
Y = df['label']

In [11]:
Y

0        CG
1        CG
2        CG
3        CG
4        CG
         ..
40414    OR
40415    CG
40416    OR
40417    CG
40418    OR
Name: label, Length: 40419, dtype: object

In [12]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()

In [13]:
Y = encoder.fit_transform(Y)

In [14]:
Y

array([0, 0, 0, ..., 1, 0, 1])

In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=0)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score,precision_score,recall_score,f1_score
import joblib

In [17]:
X_train.shape

(32335, 100)

In [18]:
Y_train.shape

(32335,)

In [19]:
model = LogisticRegression(max_iter=390)
model.fit(X_train,Y_train)

LogisticRegression(max_iter=390)

In [20]:
metric = pd.DataFrame(columns = ["Model","Accuracy","Precision","Recall","F1 Score"])

In [21]:
Y_pred = model.predict(X_test)
joblib.dump(model, 'logistic_regression_model.pkl')
metric.loc[len(metric)] = ["Logistic Regression", accuracy_score(Y_test,Y_pred), precision_score(Y_test,Y_pred), recall_score(Y_test,Y_pred), f1_score(Y_test,Y_pred)]

In [22]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier()
model.fit(X_train,Y_train)

RandomForestClassifier()

In [23]:
Y_pred = model.predict(X_test)
joblib.dump(model, 'random_forest_model.pkl')
metric.loc[len(metric)] = ["Random Forest", accuracy_score(Y_test,Y_pred), precision_score(Y_test,Y_pred), recall_score(Y_test,Y_pred), f1_score(Y_test,Y_pred)]

In [24]:
from sklearn import svm
model = svm.SVC()
model.fit(X_train,Y_train)

SVC()

In [25]:
Y_pred = model.predict(X_test)
joblib.dump(model, 'svc_model.pkl')
metric.loc[len(metric)] = ["SVM", accuracy_score(Y_test,Y_pred), precision_score(Y_test,Y_pred), recall_score(Y_test,Y_pred), f1_score(Y_test,Y_pred)]

In [26]:
metric

,Model,Accuracy,Precision,Recall,F1 Score
0,Logistic Regression,0.841910,0.840870,0.842953,0.841910
1,Random Forest,0.838570,0.873020,0.791925,0.830497
2,SVM,0.857125,0.864441,0.846668,0.855462


In [27]:
model_1 = joblib.load('logistic_regression_model.pkl')

In [28]:
from gensim.models import Word2Vec
model_w2v = Word2Vec.load("../checkpoint 1/word2vec.model")

In [29]:
def sentence_vector(sent):
    sent = [word for word in sent if word in model_w2v.wv.index_to_key]
    return np.mean(model_w2v.wv[sent],axis=0)

In [30]:
sent = "I really like this product.It has all the features which I need."
sent_w2v=[]
sent_w2v.append(sentence_vector(sent))

In [31]:
import numpy as np

In [32]:
sent_w2v = np.array(sent_w2v)

In [33]:
encoder.inverse_transform(model_1.predict(sent_w2v))[0]

'OR'

In [34]:
model_2 = joblib.load('random_forest_model.pkl')

In [35]:
encoder.inverse_transform(model_2.predict(sent_w2v))[0]

'CG'

In [36]:
model_3 = joblib.load('svc_model.pkl')

In [37]:
encoder.inverse_transform(model_3.predict(sent_w2v))[0]

'CG'